In [1]:
#pip install keras
#pip install tensorflow
#importing libraries
from keras.preprocessing import text
from tensorflow.python.keras.utils import np_utils
from keras.preprocessing import sequence
from keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [2]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [3]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [4]:
#generating (context word, target/label word) pairs
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [5]:
#model building
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2)) #to learn vector representation of words
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,))) #to cal mean of word embeddings
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15,075
Trainable params: 15,075
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 433.9504408836365

Epoch: 2 	Loss: 429.37069630622864

Epoch: 3 	Loss: 425.87414360046387

Epoch: 4 	Loss: 422.4511466026306

Epoch: 5 	Loss: 419.83328080177307



In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.020452,0.059154,-0.025390,-0.009541,0.024523,0.029350,0.040919,-0.016613,0.059066,0.026092,...,0.050368,0.028199,0.022634,-0.037605,-0.015589,0.064812,0.045815,-0.016941,-0.038934,-0.004690
networks,-0.045109,0.008303,-0.007141,0.043109,0.029085,-0.050038,-0.024587,-0.043340,0.061628,-0.025082,...,0.049578,-0.028156,0.024016,0.001615,-0.041205,0.063279,0.018135,0.035475,0.064226,0.020197
neural,-0.044200,0.013300,0.037404,-0.011776,0.009004,-0.037855,0.002572,0.014271,-0.007599,0.049251,...,0.007151,-0.010482,-0.043592,0.031567,0.017047,0.008061,-0.006041,-0.022327,0.020418,-0.005096
and,0.013451,-0.031676,-0.016381,-0.044496,-0.002798,0.013798,-0.028500,0.027068,-0.014493,0.045545,...,0.013321,0.038691,-0.031107,0.000785,-0.033502,0.022284,0.009981,-0.020101,0.015099,-0.012085
as,-0.020747,-0.047564,-0.033570,0.037867,0.029763,0.030594,-0.030457,-0.049042,0.037442,0.023020,...,0.033435,-0.001874,0.028779,0.040907,-0.034496,0.035956,0.043596,0.009393,0.003581,-0.016678


In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['and', 'where', 'applied', 'to', 'based']}

In [ ]:
# 